<a href="https://colab.research.google.com/github/aio25-mix002/m07-p7.1/blob/main/notebooks/runbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Configuration

In [6]:
!ls /kaggle/working/

ls: cannot access '/kaggle/working/': No such file or directory


In [1]:
WORKING_DIR = "/kaggle/working/"
CODE_DIR = "/kaggle/temp/src"

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download code

In [2]:
# If directory "src" not exist then clone a new one
!pwd
![ -d "{CODE_DIR}" ] || git clone --depth 1  --branch "main" "https://github.com/aio25-mix002/m07-p7.1" "{CODE_DIR}"

/content
Cloning into '/kaggle/temp/src'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 16.48 KiB | 8.24 MiB/s, done.


## Fetch the latest code

In [3]:
# Go to CODE_DIR, Fetch the latest code
%cd {CODE_DIR}
!git clean -fdx
!git status
!git pull
!pwd


/kaggle/temp/src
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Already up to date.
/kaggle/temp/src


# Data Preparation

In [4]:
!python download_data.py

Downloading...
From (original): https://drive.google.com/uc?id=141MgG4CC7XffVH32hQy7lQ0PKCafskji
From (redirected): https://drive.google.com/uc?id=141MgG4CC7XffVH32hQy7lQ0PKCafskji&confirm=t&uuid=1dd93da1-0e87-4927-8399-6eb04049339a
To: /kaggle/temp/src/HMDB51.zip
100% 3.37G/3.37G [01:03<00:00, 53.0MB/s]
Extracting HMDB51.zip...
Extraction complete.
Removed temporary file HMDB51.zip
Data is ready at: /kaggle/temp/src/hmdb51_data


# Train

In [5]:
!python train.py

Using device: cuda
Initializing datasets...
Train size: 6089 | Val size: 677
Creating model...
model.safetensors: 100% 346M/346M [00:01<00:00, 175MB/s]
Loaded pretrained weights. Missing: 132, Unexpected: 0
🚀 Compiling model with torch.compile...
Backbone FROZEN (Chỉ train SMIF & Head)

Epoch 1/10
Train:   0% 0/762 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
Val:   0% 0

In [9]:
ls -lh /kaggle/temp/src/checkpoints

total 345M
-rw-r--r-- 1 root root 345M Jan  7 06:34 best_model.pth


Save the best checkpoint to Google Drive

In [13]:
import os
import shutil
from datetime import datetime

# Define the brief note for the filename
NOTE = "vanilla" # !!! Should edit every new run
SAVE_PATH = '/content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/Artifacts/Checkpoints'


In [12]:

# Define the source path of the best model checkpoint
source_checkpoint_path = os.path.join(CODE_DIR, 'checkpoints', 'best_model.pth')

# Define your Google Drive destination folder path
# IMPORTANT: Please replace 'YOUR_GOOGLE_DRIVE_FOLDER_PATH' with the actual path to your folder in Google Drive.
# For example, it might be '/content/drive/MyDrive/MyProjectCheckpoints/'

# Ensure the Google Drive folder exists
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"Created Google Drive folder: {SAVE_PATH}")

# Generate a timestamp for the filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new filename for the checkpoint
new_checkpoint_filename = f"model_{timestamp}_{NOTE}.pth"
destination_checkpoint_path = os.path.join(SAVE_PATH, new_checkpoint_filename)

# Copy the checkpoint
try:
    shutil.copy(source_checkpoint_path, destination_checkpoint_path)
    print(f"Checkpoint successfully saved to: {destination_checkpoint_path}")
except FileNotFoundError:
    print(f"Error: Source checkpoint not found at {source_checkpoint_path}")
except Exception as e:
    print(f"An error occurred while saving the checkpoint: {e}")


Checkpoint successfully saved to: /content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/Artifacts/Checkpoints/model_20260107_081244_vanilla.pth


Reload the checkpoint (if needed)

In [14]:
# import torch
# import os

# # Define the path to the checkpoint file
# checkpoint_path = os.path.join(CODE_DIR, 'checkpoints', 'best_model.pth')

# # Check if the checkpoint file exists
# if os.path.exists(checkpoint_path):
#     # Load the checkpoint
#     loaded_checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu')) # Use 'cuda' if you want to load to GPU
#     print(f"Checkpoint loaded successfully from: {checkpoint_path}")
#     print("Keys in the loaded checkpoint:", loaded_checkpoint.keys())

#     # Example of how you might load it into a model (assuming 'model' is defined)
#     # model.load_state_dict(loaded_checkpoint['model_state_dict'])
#     # optimizer.load_state_dict(loaded_checkpoint['optimizer_state_dict'])
#     # epoch = loaded_checkpoint['epoch']
#     # loss = loaded_checkpoint['loss']
# else:
#     print(f"Error: Checkpoint not found at {checkpoint_path}")


# Submission

Inference of Test set

In [15]:
print("INFERENCE ON TEST SET")

checkpoint_path = Path('./lightweight_vit_best.pt')
print(f"Loading checkpoint from {checkpoint_path}...")
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
classes = checkpoint['classes']

model = LightweightViTForAction(num_classes=len(classes), pretrained_name=PRETRAINED_NAME).to(DEVICE)
model.load_state_dict(checkpoint['model'])
model.eval()
print(f"Model loaded (trained acc: {checkpoint['acc']:.4f})")

print("\nLoading test dataset...")
test_dataset = TestDataset(PATH_DATA_TEST, num_frames=NUM_FRAMES, frame_stride=FRAME_STRIDE, image_size=IMG_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
print(f"Test samples: {len(test_dataset)}")

NameError: name 'Path' is not defined